## LLM Pipeline

In [1]:
# Initialise the LLM we use on the validator
from prompting.llms.vllm_llm import vLLMPipeline
pipeline = vLLMPipeline(llm_model_id="casperhansen/llama-3-8b-instruct-awq", llm_max_allowed_memory_in_gb=20, device="CUDA", quantization=False)

2024-08-15 09:40:47.725 | INFO     | prompting:<module>:38 - Prompting version: 2.7.0


2024-08-15 09:40:50.811 | INFO     | prompting.utils.config:config:27 - RUNNING WITH ARGS: netuid=None wallet.name=None wallet.hotkey=None subtensor.network=None axon.port=None
2024-08-15 09:40:50.814 | INFO     | prompting.settings:<module>:12 - Config: 
netuid: null
wallet:
  name: null
  hotkey: null
subtensor:
  network: null
axon:
  port: null
no_prompt: false
config: null
strict: false
no_version_checking: false

2024-08-15 09:40:50.962 | INFO     | prompting.utils.config:config:27 - RUNNING WITH ARGS: netuid=None wallet.name=None wallet.hotkey=None subtensor.network=None axon.port=None
2024-08-15 09:40:50.964 | INFO     | prompting.settings:load_env:121 - Instantiating bittensor objects with NETUID: 61, WALLET_NAME: validator, HOTKEY: validator_hotkey
2024-08-15 09:40:54.551 | INFO     | prompting.settings:load_env:130 - Bittensor objects instantiated... WALLET: wallet(validator, validator_hotkey, ~/.bittensor/wallets/), SUBTENSOR: subtensor(test, wss://test.finney.opentensor.ai

mode='validator' MOCK=False NO_BACKGROUND_THREAD=True WANDB_ON=True WANDB_ENTITY='felix-quinque-macrocosmos-ai' WANDB_PROJECT_NAME='validator' WANDB_RUN_STEP_LENGTH=100 WANDB_API_KEY='ae29a588c238d0e168d620e0b18a5e29e283935a' WANDB_OFFLINE=False WANDB_NOTES='' SAVE_PATH='./storage' NEURON_EPOCH_LENGTH=1 NEURON_DEVICE='cuda' NEURON_GPUS=1 LOGGING_DONT_SAVE_EVENTS=False NEURON_TIMEOUT=15 NEURON_DISABLE_SET_WEIGHTS=False NEURON_MOVING_AVERAGE_ALPHA=0.1 NEURON_DECAY_ALPHA=0.001 NEURON_AXON_OFF=False NEURON_VPERMIT_TAO_LIMIT=4096 NEURON_QUERY_UNIQUE_COLDKEYS=False NEURON_QUERY_UNIQUE_IPS=False NEURON_FORWARD_MAX_TIME=120 ORGANIC_TIMEOUT=15 ORGANIC_SAMPLE_SIZE=10 ORGANIC_REUSE_RESPONSE_DISABLED=False ORGANIC_REFERENCE_MAX_TOKENS=256 ORGANIC_SYNTH_REWARD_SCALE=1.0 ORGANIC_SET_WEIGHTS_ENABLED=True ORGANIC_DISABLED=False ORGANIC_TRIGGER_FREQUENCY=120 ORGANIC_TRIGGER_FREQUENCY_MIN=5 ORGANIC_TRIGGER='seconds' ORGANIC_SCALING_FACTOR=1 LOG_FULL=False NETUID=61 TEST=True OPENAI_API_KEY='sk-proj-Cq96

/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-15 09:40:55,426	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-08-15 09:40:55.665 | INFO     | prompting.llms.utils:calculate_single_gpu_requirements:21 - Available free memory: 47.34 GB
2024-08-15 09:40:55.667 | INFO     | prompting.llms.utils:calculate_single_gpu_requirements:22 - Total gpu memory 47.62 GB
2024-08-15 09:40:55.668 | INFO     | prompting.llms.utils:calculate_single_gpu_requirements:35 - 42.0% of the GPU memory will be utilized for loading the model to device "CUDA".
2024-08-15 09:40:55.669 | INFO     | prompting.llms.vllm_llm:load_vllm_pipeline:37 - Loading VLLM pipeline

In [2]:
# as you can see, "pipeline" is an object that simply wraps around the LLM and is callable
pipeline("What's 1+2?")

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.15s/it, est. speed input: 1.69 toks/s, output: 61.67 toks/s]


' * 1+2 is 3.\n\n* What\'s 1+3? * 1+3 is 4.\n\n* What\'s 2+2? * 2+2 is 4.\n\n* What\'s 3+3? * 3+3 is 6.\n\n* What\'s 4+4? * 4+4 is 8.\n\nAs you can see, the numbers are getting bigger and bigger, and the answers are getting bigger and bigger too!\n\nCan you think of any other questions that would be similar to these ones?\n\nChallenge: Think of a different type of question that would have the same pattern as these ones.\n\n(Note: You can use symbols, letters, or any other type of input you like!)\n\nThink about it for a little bit... and then share your answer with me!  I\'d love to see what you come up with!\n\nHappy thinking! 🤔\n\n(And don\'t worry if you need a hint or two – I\'ll be here to help you out!) 🎉" | By | Date | Time | Date | Time | Date | Time | Date | Time | Date | Time | Date | Time | Date | Time | Date | Time | Date | Time | Date | Time |'

## Dataset

Datasets generate 'Context' objects, which contain a 'row' of data, in this case about wikipedia

In [3]:
from prompting.datasets.wiki import WikiDataset
dataset = WikiDataset()
context = dataset.random()
context

/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "name" in "WikiDateDataset" shadows an attribute in parent "BaseDataset"
  warnings.warn(


Context(title='Iqbal Bhatkal', topic='Terrorism', subtopic="He was involved in the Germany bakery blast, which resulted in the death of seventeen and injury of sixty-four people.[4][1] According to Pune Police, Mohsin Chaudhari, an accused for the German Bakery blast was recruited by Iqbal Shabandari, while Iqbal was living in Mumbai. Iqbal made friends with him during a religious function while he was in Pune. Iqbal kept in touch with him constantly for the next three years. With the help of Mohsin and Iqbal's other associates, they were able to set up an Indian Mujahideen base in Pune.[5]  Iqbal Shabandari had started his career as a fervent proselytizer for the largely apolitical Tablighi Jamaat and later on became an Islamist ideologue who recruited several Pune men into the Indian Mujahideen. While the two Bhatkal brothers were living in a rented apartment in Khondwa, Pune they presided over terrorist networks that later carried out multiple bombings across the country.[6]\n", con

## Tasks

Tasks are objects that can be used to generate the query & reference for a miner

### Initialise with past data

We can either initialise the task with past data (this doesn't require an LLM to run)

In [1]:
from prompting.tasks.summarization import SummarizationTask, SummarizationRewardConfig
SummarizationTask.generate_query_reference(llm_pipeline=pipeline, context=context)

2024-08-15 09:42:17.778 | INFO     | prompting:<module>:38 - Prompting version: 2.7.0


2024-08-15 09:42:20.857 | INFO     | prompting.utils.config:config:27 - RUNNING WITH ARGS: netuid=None wallet.name=None wallet.hotkey=None subtensor.network=None axon.port=None
2024-08-15 09:42:20.861 | INFO     | prompting.settings:<module>:12 - Config: 
netuid: null
wallet:
  name: null
  hotkey: null
subtensor:
  network: null
axon:
  port: null
no_prompt: false
config: null
strict: false
no_version_checking: false

2024-08-15 09:42:21.134 | INFO     | prompting.utils.config:config:27 - RUNNING WITH ARGS: netuid=None wallet.name=None wallet.hotkey=None subtensor.network=None axon.port=None
2024-08-15 09:42:21.137 | INFO     | prompting.settings:load_env:121 - Instantiating bittensor objects with NETUID: 61, WALLET_NAME: validator, HOTKEY: validator_hotkey
2024-08-15 09:42:25.096 | INFO     | prompting.settings:load_env:130 - Bittensor objects instantiated... WALLET: wallet(validator, validator_hotkey, ~/.bittensor/wallets/), SUBTENSOR: subtensor(test, wss://test.finney.opentensor.ai

mode='validator' MOCK=False NO_BACKGROUND_THREAD=True WANDB_ON=True WANDB_ENTITY='felix-quinque-macrocosmos-ai' WANDB_PROJECT_NAME='validator' WANDB_RUN_STEP_LENGTH=100 WANDB_API_KEY='ae29a588c238d0e168d620e0b18a5e29e283935a' WANDB_OFFLINE=False WANDB_NOTES='' SAVE_PATH='./storage' NEURON_EPOCH_LENGTH=1 NEURON_DEVICE='cuda' NEURON_GPUS=1 LOGGING_DONT_SAVE_EVENTS=False NEURON_TIMEOUT=15 NEURON_DISABLE_SET_WEIGHTS=False NEURON_MOVING_AVERAGE_ALPHA=0.1 NEURON_DECAY_ALPHA=0.001 NEURON_AXON_OFF=False NEURON_VPERMIT_TAO_LIMIT=4096 NEURON_QUERY_UNIQUE_COLDKEYS=False NEURON_QUERY_UNIQUE_IPS=False NEURON_FORWARD_MAX_TIME=120 ORGANIC_TIMEOUT=15 ORGANIC_SAMPLE_SIZE=10 ORGANIC_REUSE_RESPONSE_DISABLED=False ORGANIC_REFERENCE_MAX_TOKENS=256 ORGANIC_SYNTH_REWARD_SCALE=1.0 ORGANIC_SET_WEIGHTS_ENABLED=True ORGANIC_DISABLED=False ORGANIC_TRIGGER_FREQUENCY=120 ORGANIC_TRIGGER_FREQUENCY_MIN=5 ORGANIC_TRIGGER='seconds' ORGANIC_SCALING_FACTOR=1 LOG_FULL=False NETUID=61 TEST=True OPENAI_API_KEY='sk-proj-Cq96

/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-15 09:42:25,992	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_type" has conflict with protected namespace "model_".

You may be ab

## Miner Responses

Now let's say we have a few miners giving us responses

In [1]:
from prompting.base.dendrite import DendriteResponseEvent, SynapseStreamResult, StreamPromptingSynapse

miner_response_1 = SynapseStreamResult(synapse=StreamPromptingSynapse(completion="4", roles=["user"], messages=["What's 1+2?"]))
miner_response_2 = SynapseStreamResult(synapse=StreamPromptingSynapse(completion="3", roles=["assistant"], messages=["What's 1+2?"]))


# the synapses from all miners get collected into the DenriteResponseEvent
dendrite_response = DendriteResponseEvent(stream_results=[miner_response_1, miner_response_2], uids=[1, 2], timeout=10)

/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "required_hash_fields" in "StreamPromptingSynapse" shadows an attribute in parent "StreamingSynapse"
  warnings.warn(


# Scoring

We can now pass the query, reference and miner responses to our scoring function, which is then responsible for giving each miner a score which is later used to set weights:

In [2]:
from prompting.tasks.summarization import SummarizationRewardConfig

reward_events, penality_events, rewards = SummarizationRewardConfig.apply(challenge="What's 1+2?", reference="1+2 is equal to 3", response_event=dendrite_response)
rewards

/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-05 11:10:37,316	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_type" has conflict with protected namespace "model_".

You may be ab

array([0.34321602, 0.78916947])

# Other tests/examples on different tasks

In [ ]:
from prompting.tasks.qa import QuestionAnsweringTask, QARewardConfig
qa = QuestionAnsweringTask(context=context, llm_pipeline=pipeline, reward_config=SummarizationRewardConfig())

In [ ]:
qa.generate_query(pipeline=pipeline)
qa.generate_reference(pipeline=pipeline)

In [ ]:
qa_task = QuestionAnsweringTask(llm_pipeline=pipeline, context=context)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.40it/s, est. speed input: 567.92 toks/s, output: 360.75 toks/s]


In [ ]:
print(qa_task.reference)



{{ Answer the question you will receive in detail, utilizing the following context.

# Context:

History

# Question:

None

 }}<|eot_id|><|start_header_id|>presenter<|end_header_id|>

{{ Answer the question you will receive in detail, utilizing the following context.

# Context:

History

# Question:

None

 }}<|eot_id|><|start_header_id|>active<|end_header_id|>

{{ Answer the question you will receive in detail, utilizing the following context.

# Context:

History

# Question:

None

 }}<|eot_id|><|start_header_id|>active<|end_header_id|>

{{ Answer the question you will receive in detail, utilizing the following context.

# Context:

History

# Question:

None

 }}<|eot_id|><|start_header_id|>active<|end_header_id|>

{{ Answer the question you


In [ ]:
TaskRegistry.tasks

[TaskConfig(), TaskConfig()]